Aim: To detemine the value of $\pi$ by Monte Carlo method.

In [ ]:
!pip install pycuda

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import numpy as np
from sympy import Rational

In [ ]:
import threading

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-f_mj2zrz
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-f_mj2zrz
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=62828a59383c4cf05aec40b47dcc5c5277b4692075fa06d09c15493319e74f53
  Stored in directory: /tmp/pip-ephem-wheel-cache-os0lj_0m/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
ker = SourceModule(no_extern_c=True ,source='''

#include <curand_kernel.h>
#define _PYTHAG(a,b)  (a*a + b*b)
#define ULL  unsigned long long
extern "C" {
__global__ void estimate_pi(ULL iters, ULL * hits)
{
	curandState cr_state;
     
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	curand_init( (ULL)  clock() + (ULL) tid, (ULL) 0, \
	(ULL) 0, &cr_state);
	float x, y;
 
	for(ULL i=0; i < iters; i++)
	{ 
		 x = curand_uniform(&cr_state);
		 y = curand_uniform(&cr_state);
		 
		 
		 if(_PYTHAG(x,y) <= 1.0f)
			 hits[tid]++;
	}
 
 return;
}
}// (End of 'extern "C"' here)
''')

In [ ]:
%%time
pi_ker = ker.get_function("estimate_pi")

threads_per_block = 32
blocks_per_grid = 512 

total_threads = threads_per_block * blocks_per_grid

hits_d = gpuarray.zeros((total_threads,),dtype=np.uint64)

iters = 2**24   

pi_ker(np.uint64(iters), hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

total_hits = np.sum( hits_d.get()  )
total = np.uint64(total_threads) * np.uint64(iters)

est_pi_symbolic =  Rational(4)*Rational(int(total_hits), int(total) )

est_pi = np.float(est_pi_symbolic.evalf())

print ("Our Monte Carlo estimate of Pi is : %s" % est_pi)
print ("NumPy's Pi constant is: %s " % np.pi)

print ("Our estimate passes NumPy's 'allclose' : %s" % np.allclose(est_pi, np.pi))

Our Monte Carlo estimate of Pi is : 3.1415968635410536
NumPy's Pi constant is: 3.141592653589793 
Our estimate passes NumPy's 'allclose' : True
CPU times: user 12.2 s, sys: 8.68 s, total: 20.8 s
Wall time: 20.9 s


## **Aim: To determine the volume of N-Dimensional Hyper-sphere**

In [ ]:
import math, random

In [ ]:
def isPointInCircle(x, y, Cx, Cy, radius):
    return math.sqrt((x - Cx)**2 + (y - Cy)**2) < radius

In [ ]:
def approximateCircleArea(radius, numberOfPoints):
    squareSide = radius*2
    Cx = radius
    Cy = radius
 
    pointsInside = 0
    for i in range(numberOfPoints):
        x = random.random()*squareSide
        y = random.random()*squareSide
 
        if (isPointInCircle(x, y, Cx, Cy, radius)):
            pointsInside = pointsInside + 1
 
    return pointsInside / numberOfPoints * squareSide**2

In [ ]:
%%time
d = 2 # dimensions
n = 10000000 # number of hits
approximateCircleArea(1,n)

3.1413448

## **3-D**

In [ ]:
ker = SourceModule(no_extern_c=True ,source='''

#include <curand_kernel.h>
#define _PYTHAG(a,b,c)  (a*a + b*b + c*c)
#define ULL  unsigned long long
extern "C" {
__global__ void estimate_vol(ULL iters, ULL * hits)
{
	curandState cr_state;
     
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	curand_init( (ULL)  clock() + (ULL) tid, (ULL) 0, \
	(ULL) 0, &cr_state);
	float x, y, z;
 
	for(ULL i=0; i < iters; i++)
	{ 
		 x = curand_uniform(&cr_state);
		 y = curand_uniform(&cr_state);
		 z = curand_uniform(&cr_state);
		 
		 if(_PYTHAG(x,y,z) <= 1.0f)
			 hits[tid]++;
	}
 
 return;
}
}// (End of 'extern "C"' here)
''')

In [ ]:
%%time
pi_ker = ker.get_function("estimate_vol")

threads_per_block = 32
blocks_per_grid = 512 

total_threads = threads_per_block * blocks_per_grid

hits_d = gpuarray.zeros((total_threads,),dtype=np.uint64)

iters = 2**24   

pi_ker(np.uint64(iters), hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

total_hits = np.sum( hits_d.get()  )
total = np.uint64(total_threads) * np.uint64(iters)

est_pi_symbolic =  Rational(2**3)*Rational(int(total_hits), int(total) )

est_pi = np.float(est_pi_symbolic.evalf())

print ("Our Monte Carlo estimate of Pi is : %s" % est_pi)
print ("NumPy's Pi constant is: ", (4.0/3.0)*np.pi)

print ("Our estimate passes NumPy's 'allclose' :," , np.allclose(est_pi, (4/3)*np.pi))

Our Monte Carlo estimate of Pi is : 4.188789140112931
NumPy's Pi constant is:  4.1887902047863905
Our estimate passes NumPy's 'allclose' :, True
CPU times: user 13 s, sys: 9.21 s, total: 22.2 s
Wall time: 22.2 s


In [ ]:
ker = SourceModule(no_extern_c=True ,source='''

#include <curand_kernel.h>
#define _PYTHAG(a,b,c)  (a*a + b*b + c*c)
#define ULL  unsigned long long
extern "C" {
__global__ void estimate_vol_plus(ULL iters, ULL * hits)
{
	curandState cr_state;
     
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	curand_init( (ULL)  clock() + (ULL) tid, (ULL) 0, \
	(ULL) 0, &cr_state);
	float x, y, z, r;
  r = 1.1; 
	for(ULL i=0; i < iters; i++)
	{ 
		 x = r*curand_uniform(&cr_state);
		 y = r*curand_uniform(&cr_state);
		 z = r*curand_uniform(&cr_state);
		 
		 if(_PYTHAG(x,y,z) <= r*r)
			 hits[tid]++;
	}
 
 return;
}
}// (End of 'extern "C"' here)
''')


In [ ]:
%%time
pi_ker = ker.get_function("estimate_vol_plus")
r = 1.1
threads_per_block = 32
blocks_per_grid = 512 

total_threads = threads_per_block * blocks_per_grid

hits_d = gpuarray.zeros((total_threads,),dtype=np.uint64)

iters = 2**19   

pi_ker(np.uint64(iters), hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

total_hits = np.sum( hits_d.get()  )
total = np.uint64(total_threads) * np.uint64(iters)

est_pi_symbolic =  Rational((2*r)**3)*Rational(int(total_hits), int(total) )

est_pi = np.float(est_pi_symbolic.evalf())

print ("Our Monte Carlo estimate of Pi is : %s" % est_pi)
print ("NumPy's Pi constant is: ", (4/3)*np.pi*(r**3))

print ("Our estimate passes NumPy's 'allclose' :" , np.allclose(est_pi, (4/3)*np.pi*(r**3)))

Our Monte Carlo estimate of Pi is : 5.575321346065962
NumPy's Pi constant is:  5.575279762570688
Our estimate passes NumPy's 'allclose' : True
CPU times: user 374 ms, sys: 360 ms, total: 734 ms
Wall time: 741 ms


In [ ]:
print('Ratio of change in volume to change in radius: ',(5.57528254103879-4.188789140112931)/0.1)

Ratio of change in volume to change in radius:  13.864934009258594


## **N-D**

In [ ]:
ker = SourceModule(no_extern_c=True ,source='''

#include <curand_kernel.h>
#define ULL  unsigned long long
extern "C" {
__global__ void estimate_vol( ULL iters,ULL dim, ULL r, ULL * hits)
{
	curandState cr_state;
     
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	curand_init( (ULL)  clock() + (ULL) tid, (ULL) 0, \
	(ULL) 0, &cr_state);
	float x, y;
	for(ULL i=0; i < iters; i++)
	{ 
    y = 0;
    for(int j = 0; j < dim; j++)
    {
      x = r*curand_uniform(&cr_state);
      y += x*x;		 
    }
		 if(y <= r*r)
			 hits[tid]++;
	}
 
 return;
}
}// (End of 'extern "C"' here)
''')


In [ ]:
import math # for factorial calculation

In [ ]:
def fac(n): # only for gamma function calculation
  if (n > 1):
    return (n-1)*fac(n-1);
  else:
    return 1;     

In [ ]:
%%time
r = 1
N = 1
_3N = 3*N
pi_ker = ker.get_function("estimate_vol")
threads_per_block = 32
blocks_per_grid = 512 

total_threads = threads_per_block * blocks_per_grid

hits_d = gpuarray.zeros((total_threads,),dtype=np.uint64)

iters = 2**30   

pi_ker(np.uint64(iters), np.uint64(_3N), np.uint64(r), hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

total_hits = np.sum( hits_d.get()  )
total = np.uint64(total_threads) * np.uint64(iters)

est_pi_symbolic =  Rational((2*r)**(_3N))*Rational(int(total_hits), int(total) )

est_pi = np.float(est_pi_symbolic.evalf())

print ("Our Monte Carlo estimate of volume of radius ",r," is : ", est_pi )
if((_3N)//2 == 0):
  print ("NumPy's output is: ", (( (np.pi)**((_3N)/2) )/(math.factorial(_3N*0.5))) * (r**(_3N)))
  print ("Our estimate passes NumPy's 'allclose' :" , np.allclose(est_pi, (( (np.pi)**((_3N)/2) )/(math.factorial(_3N*0.5))) * (r**(_3N))))
else:
  print ("NumPy's output is: ", (( (np.pi)**((_3N - 1)*0.5) )/(fac(_3N*0.5 + 1))) * (r**(_3N)))
  print ("Our estimate passes NumPy's 'allclose' :" , np.allclose(est_pi, (( (np.pi)**((_3N - 1)*0.5) )/(fac(_3N*0.5))) * (r**(_3N)))  )     

Our Monte Carlo estimate of volume of radius  1  is :  4.18878991933434
NumPy's output is:  4.1887902047863905
Our estimate passes NumPy's 'allclose' : False
CPU times: user 30min 14s, sys: 26min 5s, total: 56min 19s
Wall time: 56min 22s


In [ ]:
%%time

N = 100
r = 1
_3N = 3*N

iters = 2**29
hits = 0
for i in range(iters):
  y = 0
  for j in range(_3N):
    x = r*np.random.rand()
    y += x**2
  if(y <= r**2):
    hits += 1
est = (2**(_3N))*(hits/iters)
print(est)
print((4/3)*np.pi*(r**3))
print ("Our estimate passes NumPy's 'allclose' :" , np.allclose(est, (4/3)*np.pi*(r**3)))  

4.188625782728195
4.1887902047863905
Our estimate passes NumPy's 'allclose' : False
CPU times: user 22min 45s, sys: 503 ms, total: 22min 45s
Wall time: 22min 47s


In [ ]:
n = 10

x = np.random.randn(3,2**n)

iters = 2**24






In [ ]:
num_arrays = 10
array_len = 1024**2
kernel_code = """ 
__global__ void mult_ker(float * in_array, float * out_array, unsigned long long array_len)
{
    int thd = blockIdx.x*blockDim.x + threadIdx.x;
    for(int j=0; j < array_len; j++)
    {
      int i = j * blockDim.x + thd;
      for(int k = 0; k < 50; k++)
      {
        if(in_array[])
        out_array[i] *= 2.0;
         array[i] /= 2.0;
     }
 }
}
"""

In [ ]:
ker = SourceModule(no_extern_c=True ,source='''
#include <curand_kernel.h>
#define ULL  unsigned long long
extern "C" {
__global__ void estimate_vol( ULL iters,ULL dim, ULL r, ULL * hits)
{
	curandState cr_state;
     
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	curand_init( (ULL)  clock() + (ULL) tid, (ULL) 0, \
	(ULL) 0, &cr_state);
	float x, y;
	for(ULL i=0; i < iters; i++)
	{ 
    y = 0;
    for(int j = 0; j < dim; j++)
    {
      x = r*curand_uniform(&cr_state);
      y += x*x;		 
    }
		 if(y <= r*r)
			 hits[tid]++;
	}
 
 return;
}
}// (End of 'extern "C"' here)
''')


# **Integral points**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 10 # number of dimensions
a = np.arange(N, dtype=np.uint8)
for i in range(N-1):
  a = np.column_stack((a,np.arange(N)))
a = a.T

In [ ]:
def Recurse(y, number, n=[]): 
  if (number > 1):
    Recurse( y, number - 1)
  else:
    for x in range(y):
      print()
      

In [ ]:
def Recurse(y, number, n=[]): 
  if (number > 1):
    Recurse( y, number - 1, n.append() )
  else:
    for x in range(y):
      if(np.sum(np.array([i**2 for i in n])) <= N**2):
        hits += 0.25
      

In [ ]:
kernel_code = SourceModule("""
__global__ void kernel(int * out)
{
  int idx = blockIdx.x*blockDim.x + threadIdx.x;
  if(idx < 10)
  {
    out[0] += 1;    
  }
}
""")

In [ ]:
kernel_code = SourceModule("""
__global__ void kernel_sleep(int * out)
{
  int idx = blockIdx.x*blockDim.x + threadIdx.x;
  clock_t start = clock();
  clock_t now;
  for (;;) 
  {
    now = clock();
    clock_t cycles = now > start ? now - start : now + (0xffffffff - start);
    if (cycles >= 1000*(1 + idx)) 
    {
      if(idx < 10)
      {
        out[0] += 1;    
      }
    }
  }
}
""")

In [ ]:
h_ker = kernel_code.get_function("kernel")
threads_per_block = 16
blocks_per_grid = 1 
total_threads = threads_per_block * blocks_per_grid
hits_d = gpuarray.zeros((1,),dtype=np.uint64)
h_ker(hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

[0]


In [ ]:
h_ker = kernel_code.get_function("kernel_sleep")
threads_per_block = 16
blocks_per_grid = 1 
total_threads = threads_per_block * blocks_per_grid
hits_d = gpuarray.zeros((1,),dtype=np.uint64)
h_ker(hits_d, grid=(blocks_per_grid,1,1), block=(threads_per_block,1,1))

## **Number of integral points inside a hypersphere of N-dimensions (SOLVED)**

In [ ]:
%%time
%%cu

#include<iostream>
#include<cmath>
using namespace std;
#define _R1 (9);
#define _R2 (10);
#define _N (10);
#define _DIM (3);
#define _THRDS_P_BLK (32);
#define _MAX_DIM_PER_LOOP (6);

__global__ void kernel(int* x, int* y, int off)
{
    int n = _N;
    const int dim = _DIM;
    int idx = blockIdx.x*blockDim.x + threadIdx.x;
    int arr[dim] = {0};
    int max_dim_per_loop = _MAX_DIM_PER_LOOP;
    unsigned long long int index = idx + off*pow(n,max_dim_per_loop);
    for(int i=0;i<dim; i++)
    {
        arr[i] = index/pow(n,dim-1-i);
        index -= arr[i]*pow(n,dim-1-i);
    }
    
    int sum_o_sq = 0;
    for(int j=0; j<dim; j++)
    {
        sum_o_sq += pow(arr[j],2);
    }
    int R1 = _R1;
    int R2 = _R2;
    
    if(sum_o_sq <= pow(R1,2))
    {
        x[idx] += 1;
    }
    if(sum_o_sq <= pow(R2,2))
    {
        y[idx] += 1;
    }
}

__host__ int main()
{
    int ref_N = _N;
    int dim = _DIM;
    const int max_dim_per_loop = _MAX_DIM_PER_LOOP;
    int n = 0;
    if(dim > max_dim_per_loop)
    {
        n = dim - max_dim_per_loop;   
    }
    unsigned long long int N = pow(ref_N,dim-n);
    n = ref_N*n;
    int h_offset = 0;
    int hits_1[N] = {0};
    int hits_2[N] = {0};
    int * d_hits_1;
    int * d_hits_2;
    int sz = sizeof(hits_1)/sizeof(*hits_1);
    sz *= sizeof(int);
    cout << "GPU memory usage: " << sz << " bytes" << endl;
    cudaMalloc((void**)&d_hits_1, sz);
    cudaMalloc((void**)&d_hits_2, sz);
    cudaMemcpy(d_hits_1,&hits_1,sz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_hits_2,&hits_2,sz, cudaMemcpyHostToDevice);
    int threads_per_block = _THRDS_P_BLK;
    int block_per_grid = N/threads_per_block;
    kernel<<< block_per_grid,threads_per_block >>>(d_hits_1, d_hits_2,h_offset); 
    n -= 1;
    h_offset += 1;
    while(n >= 0)
    {
        kernel<<< block_per_grid,threads_per_block >>>(d_hits_1, d_hits_2,h_offset); 
        n -= 1;
        h_offset += 1;
    }
    cudaMemcpy(&hits_1,d_hits_1,sz, cudaMemcpyDeviceToHost);
    cudaMemcpy(&hits_2,d_hits_2,sz, cudaMemcpyDeviceToHost);
    int h1 = 0;
    int h2 = 0;
    for(int i=0;i<N;i++)
    {
        h1 += hits_1[i];
        h2 += hits_2[i];
    }
    
    cudaFree(d_hits_1);
    cudaFree(d_hits_2);
    
    int R1 = _R1;
    int R2 = _R2;
    float V1 = (1.0/pow(2,dim)) + (pow(R1,dim)/pow(2.0,dim-1)) + (h1*pow(2.0,dim) - pow(R1,dim) - 1);
    float V2 = (1.0/pow(2,dim)) + (pow(R2,dim)/pow(2.0,dim-1)) + (h2*pow(2.0,dim) - pow(R2,dim) - 1);
    cout << "Number of points inside/on the hypersphere of radius, R1: " << h1 << endl;
    cout << "Number of points inside/on the hypersphere of radius, R2: " << h2 << endl;
    cout << "Volume of the hypersphere of radius R1 computed by counting the number of integral points inside (and on) the same: " << V1 << endl;
    cout << "Volume of the hypersphere of radius R2 computed by counting the number of integral points inside (and on) the same: " << V2 << endl;
    return 0;
}

GPU memory usage: 4000 bytes
Number of points inside/on the hypersphere of radius, R1: 486
Number of points inside/on the hypersphere of radius, R2: 645
Volume of the hypersphere of radius R1 computed by counting the number of integral points inside (and on) the same: 3340.38
Volume of the hypersphere of radius R2 computed by counting the number of integral points inside (and on) the same: 4409.12

CPU times: user 1.83 ms, sys: 13.8 ms, total: 15.7 ms
Wall time: 1.61 s


In [ ]:
%%time
%%cu

#include<iostream>
#include<cmath>
using namespace std;
#define _R1 (9);
#define _R2 (10);
#define _N (10);
#define _THRDS_P_BLK (32);
#define _MAX_DIM_PER_LOOP (6);

__global__ void kernel(int* x, int* y, int off)
{
    int n = _N;
    int idx = blockIdx.x*blockDim.x + threadIdx.x;
    int arr[n] = {0};
    int max_dim_per_loop = _MAX_DIM_PER_LOOP;
    unsigned long long int index = idx + off*pow(n,max_dim_per_loop);
    for(int i=0;i<n; i++)
    {
        arr[i] = index/pow(10,n-1-i);
        index -= arr[i]*pow(n,dim-1-i);
    }
    
    int sum_o_sq = 0;
    for(int j=0; j<dim; j++)
    {
        sum_o_sq += pow(arr[j],2);
    }
    int R1 = _R1;
    int R2 = _R2;
    
    if(sum_o_sq <= pow(R1,2))
    {
        x[idx] += 1;
    }
    if(sum_o_sq <= pow(R2,2))
    {
        y[idx] += 1;
    }
}

__host__ int main()
{
    int ref_N = _N;
    int dim = _DIM;
    const int max_dim_per_loop = _MAX_DIM_PER_LOOP;
    int n = 0;
    if(dim > max_dim_per_loop)
    {
        n = dim - max_dim_per_loop;   
    }
    unsigned long long int N = pow(ref_N,dim-n);
    n = ref_N*n;
    int h_offset = 0;
    int hits_1[N] = {0};
    int hits_2[N] = {0};
    int * d_hits_1;
    int * d_hits_2;
    int sz = sizeof(hits_1)/sizeof(*hits_1);
    sz *= sizeof(int);
    cout << "GPU memory usage: " << sz << " bytes" << endl;
    cudaMalloc((void**)&d_hits_1, sz);
    cudaMalloc((void**)&d_hits_2, sz);
    cudaMemcpy(d_hits_1,&hits_1,sz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_hits_2,&hits_2,sz, cudaMemcpyHostToDevice);
    int threads_per_block = _THRDS_P_BLK;
    int block_per_grid = N/threads_per_block;
    kernel<<< block_per_grid,threads_per_block >>>(d_hits_1, d_hits_2,h_offset); 
    n -= 1;
    h_offset += 1;
    while(n >= 0)
    {
        kernel<<< block_per_grid,threads_per_block >>>(d_hits_1, d_hits_2,h_offset); 
        n -= 1;
        h_offset += 1;
    }
    cudaMemcpy(&hits_1,d_hits_1,sz, cudaMemcpyDeviceToHost);
    cudaMemcpy(&hits_2,d_hits_2,sz, cudaMemcpyDeviceToHost);
    int h1 = 0;
    int h2 = 0;
    for(int i=0;i<N;i++)
    {
        h1 += hits_1[i];
        h2 += hits_2[i];
    }
    
    cudaFree(d_hits_1);
    cudaFree(d_hits_2);
    
    int R1 = _R1;
    int R2 = _R2;
    float V1 = (1.0/pow(2,dim)) + (pow(R1,dim)/pow(2.0,dim-1)) + (h1*pow(2.0,dim) - pow(R1,dim) - 1);
    float V2 = (1.0/pow(2,dim)) + (pow(R2,dim)/pow(2.0,dim-1)) + (h2*pow(2.0,dim) - pow(R2,dim) - 1);
    cout << "Number of points inside/on the hypersphere of radius, R1: " << h1 << endl;
    cout << "Number of points inside/on the hypersphere of radius, R2: " << h2 << endl;
    cout << "Volume of the hypersphere of radius R1 computed by counting the number of integral points inside (and on) the same: " << V1 << endl;
    cout << "Volume of the hypersphere of radius R2 computed by counting the number of integral points inside (and on) the same: " << V2 << endl;
    return 0;
}

## **Previous versions**

In [ ]:
%%time
%%cu

#include<iostream>
#include<cmath>
using namespace std;
#define _R1 (9);
#define _R2 (10);
#define _N (10);
#define _DIM (6);
#define _THRDS_P_BLK (1024);

__host__ unsigned int nextPowerOf2(unsigned int n)  
{  
    unsigned count = 0;  
      
    // First n in the below condition  
    // is for the case where n is 0  
    if (n && !(n & (n - 1)))  
        return n;  
      
    while( n != 0)  
    {  
        n >>= 1;  
        count += 1;  
    }  
      
    return 1 << count;  
}  
 
__global__ void kernel(int* x, int* y)
{
    int n = _N;
    const int dim = _DIM;
    int idx = blockIdx.x*blockDim.x + threadIdx.x;
    int arr[dim] = {0};
    int sub = 0;
    int index = idx;
    for(int i=0;i<dim; i++)
    {
        arr[i] = index/pow(n,dim-1-i);
        index -= arr[i]*pow(n,dim-1-i);
    }
    
    int sum_o_sq = 0;
    for(int j=0; j<dim; j++)
    {
        sum_o_sq += pow(arr[j],2);
    }
    int R1 = _R1;
    int R2 = _R2;
    
    if(sum_o_sq <= pow(R1,2))
    {
        x[idx] = 1;
    }
    if(sum_o_sq <= pow(R2,2))
    {
        y[idx] = 1;
    }
}

__host__ int main()
{
    int N = _N;
    int dim = _DIM;
    N = pow(N,dim);
    //N = nextPowerOf2(N);
    int hits_1[N] = {0};
    int hits_2[N] = {0};
    int * d_hits_1;
    int * d_hits_2;
    int sz = sizeof(hits_1)/sizeof(*hits_1);
    sz *= sizeof(int);
    cout << sz << endl;
    cudaMalloc((void**)&d_hits_1, sz);
    cudaMalloc((void**)&d_hits_2, sz);
    cudaMemcpy(d_hits_1,&hits_1,sz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_hits_2,&hits_2,sz, cudaMemcpyHostToDevice);
    int threads_per_block = _THRDS_P_BLK;
    int block_per_grid = N/threads_per_block;
    kernel<<< block_per_grid,threads_per_block >>>(d_hits_1, d_hits_2); 
    cudaMemcpy(&hits_1,d_hits_1,sz, cudaMemcpyDeviceToHost);
    cudaMemcpy(&hits_2,d_hits_2,sz, cudaMemcpyDeviceToHost);
    int h1 = 0;
    int h2 = 0;
    for(int i=0;i<N;i++)
    {
        h1 += hits_1[i];
        h2 += hits_2[i];
    }
    
    cudaFree(d_hits_1);
    cudaFree(d_hits_2);
    
    cout << "Number of points inside/on the hypersphere of radius, R1: " << h1 << endl;
    cout << "Number of points inside/on the hypersphere of radius, R2: " << h2 << endl;
    
    return 0;
}

4000000
Number of points inside/on the hypersphere of radius, R1: 81586
Number of points inside/on the hypersphere of radius, R2: 145132

CPU times: user 531 µs, sys: 11.4 ms, total: 11.9 ms
Wall time: 1.67 s


In [ ]:
%%time
%%cu

#include<iostream>
#include<cmath>
using namespace std;

int main()
{
    int n = 100;
    const int dim = 3;
    int h1 = 0;
    int h2 = 0;
    int idx_len = pow(n,dim);
    unsigned long idx[idx_len] = {0};
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<n;j++)
        {
            for(int k=0;k<n;k++)
            {
                if(pow(i,2) + pow(j,2) + pow(k,2) <= pow(9,2))
                {
                    h1 += 1;
                }
                if(pow(i,2) + pow(j,2) + pow(k,2) <= pow(10,2))
                {
                    h2 += 1;
                }                    
            }
        }
    }  
    cout << "Number of points inside hypersphere of radius, R1: " << h1 << endl;
    cout << "Number of points inside hypersphere of radius, R2: " << h2 << endl;
    
    return 0;
}

Number of points inside hypersphere of radius, R1: 486
Number of points inside hypersphere of radius, R2: 648

CPU times: user 1.79 ms, sys: 11.2 ms, total: 13 ms
Wall time: 1.62 s


In [ ]:
%%cu

#include<iostream>
#include<cmath>
using namespace std;

int main()
{
    int n = 10;
    const int dim = 3;
    int h1 = 0;
    int h2 = 0;
    int idx_len = pow(n,dim);
    unsigned long idx[idx_len] = {0};
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<n;j++)
        {
            for(int k=0;k<n;k++)
            {
                idx[i*n*n + j*n + k] = i*n*n + j*n + k;
            }
        }
    }  
    for(int i=0;i<idx_len; i++)
    {
        int a = idx[i]/pow(n,2);
        idx[i] -= a*pow(n,2);
        int b = idx[i]/pow(n,1);
        idx[i] -= b*pow(n,1);
        int c = idx[i];
        if(pow(a,2) + pow(b,2) + pow(c,2) <= pow(9,2))
        {
            h1 += 1;
        }
        if(pow(a,2) + pow(b,2) + pow(c,2) <= pow(10,2))
        {
            h2 += 1;
        }
    }
    cout << "Number of points inside hypersphere of radius, R1: " << h1 << endl;
    cout << "Number of points inside hypersphere of radius, R2: " << h2 << endl;
    
    return 0;
}

Number of points inside hypersphere of radius, R1: 486
Number of points inside hypersphere of radius, R2: 645



In [ ]:

%%cu
__host__ unsigned int nextPowerOf2(unsigned int n)  
{  
    unsigned count = 0;  
      
    // First n in the below condition  
    // is for the case where n is 0  
    if (n && !(n & (n - 1)))  
        return n;  
      
    while( n != 0)  
    {  
        n >>= 1;  
        count += 1;  
    }  
      
    return 1 << count;  
}  
